In [247]:
import os
import pandas as pd
import numpy as np
import csv 
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [5]:
os.chdir("C:\\HTW\\Advance Data Analytics")

## Reading the Data

In [ ]:
# Secom Data 
url1 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
r1 = rq.get(url1, allow_redirects=True)
open('secomdata.csv', 'wb').write(r1.content)

secom_data = pd.read_csv("secom.csv", sep = " ", header = None)

In [6]:
#Secom Label Data
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'
r = rq.get(url, allow_redirects=True)
open('secomlabels.csv', 'wb').write(r.content)

secom_label = pd.read_csv("secom_labels.csv", sep = " ", header = None)

## Reformating Data

In [7]:
# Renaming secom dataset columns into feature names 

secom_data.columns = ["feature"+ str(i+1) for i in range(len(secom_data.columns))]
secom_label.columns = ["Status", "Timestamp"]

In [ ]:
# Splitting Timestamp into Date and Time
secom_label["Date"] = pd.to_datetime(secom_label["Timestamp"]).dt.date
secom_label["Time"] = pd.to_datetime(secom_label["Timestamp"]).dt.time
secom_label.drop("Timestamp", axis = 1, inplace = True)

In [233]:
df_merge = pd.concat([secom_label, secom_data], axis=1)
df_merge

,Status,Date,Time,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature581,feature582,feature583,feature584,feature585,feature586,feature587,feature588,feature589,feature590
0,-1,2008-07-19,11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,-1,2008-07-19,12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,1,2008-07-19,13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,-1,2008-07-19,14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,-1,2008-07-19,15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,-1,2008-10-16,15:13:00,2899.41,2464.36,2179.7333,3085.3781,1.4843,100.0,82.2467,...,0.0047,203.1720,0.4988,0.0143,0.0039,2.8669,0.0068,0.0138,0.0047,203.1720
1563,-1,2008-10-16,20:49:00,3052.31,2522.55,2198.5667,1124.6595,0.8763,100.0,98.4689,...,NaN,NaN,0.4975,0.0131,0.0036,2.6238,0.0068,0.0138,0.0047,203.1720
1564,-1,2008-10-17,05:26:00,2978.81,2379.78,2206.3000,1110.4967,0.8236,100.0,99.4122,...,0.0025,43.5231,0.4987,0.0153,0.0041,3.0590,0.0197,0.0086,0.0025,43.5231
1565,-1,2008-10-17,06:01:00,2894.92,2532.01,2177.0333,1183.7287,1.5726,100.0,98.7978,...,0.0075,93.4941,0.5004,0.0178,0.0038,3.5662,0.0262,0.0245,0.0075,93.4941


## Function Defination

In [118]:
# Min Max Scaling 
def scaler(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler.transform(data)

In [255]:
# KNN Imputation
def knn_imputer(data):
    global knn
    knn = KNNImputer(n_neighbors=5)
    knn.fit(data)
    return knn.transform(data)

In [256]:
# MICE Imputation
def mice_imputer(data):
    lr = LinearRegression()
    global mice
    mice = IterativeImputer(estimator = lr, verbose = 2, max_iter = 5, tol = 1e-10, imputation_order = "roman")
    mice.fit(data)
    return mice.transform(data)

In [249]:
# Feature reduction having missing value more than 55 percent
def trim_df(data):
    threshold = int(0.55*data.shape[0])
    df4 = pd.DataFrame(data.isna().sum())
    rem_col = list(df4[df4[0]>=threshold].index)
    return data.drop(columns = rem_col, axis = 1)

## Split and Training the Model

In [260]:
# Removing Missing value columns
df_merge = trim_df(df_merge)

In [261]:
# Train Test split of faulty and non-faulty dataset
faulty = df_merge[df_merge["Status"] == 1]
nfaulty = df_merge[df_merge["Status"] == -1]

faulty_train, faulty_test = train_test_split(faulty, test_size = 0.2)
nfaulty_train, nfaulty_test = train_test_split(nfaulty, test_size = 0.2)

test_data = pd.concat([faulty_test, nfaulty_test])
train_data = pd.concat([faulty_train, nfaulty_test])


In [262]:
# KNN_Imputing on datasets
X_train_knn = knn_imputer(X_train)

# KNN_Imputing on X_Test based on fit(pattern) of X_train
X_test_knn = knn.transform(X_test)

In [258]:
# MICE_Imputing 
X_train_mice = mice_imputer(X_train)

# MICE imputing on X_Test based on fit of X_train
X_test_mice = mice.transform(X_test)

[IterativeImputer] Completing matrix with shape (1253, 590)
[IterativeImputer] Ending imputation round 1/5, elapsed time 35.15
[IterativeImputer] Change: 0.0, scaled tolerance: 3.7943000000000002e-06 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (1253, 590)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.01
[IterativeImputer] Completing matrix with shape (314, 590)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.00
